In [ ]:
pip install tensorflow

In [ ]:
pip install tensorflow_hub

In [63]:
import tensorflow as tf
import tensorflow_hub as hub

import matplotlib.pyplot as plt
import pandas as pd

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

Version:  2.3.1
Eager mode:  True
Hub version:  0.9.0
GPU is NOT AVAILABLE


In [64]:
ls

Error_analysis/                Untitled.ipynb
Is-Humor_NN.ipynb              Untitled1.ipynb
Naive_Bayes_week5.ipynb        abc.txt
Naive_Bayes_week5_nisha.ipynb  eval_data.csv
Predict_is_humor.ipynb         new_dff-2.xlsx
TASk_1a.csv                    new_dff.csv
TASk_1a_LR.csv                 public_dev.csv
TASk_1a_LR.zip                 train.csv
TF_notebook.ipynb              week_4_NN.ipynb


In [66]:
dataset = pd.read_csv('train.csv')
print(dataset.head())

#only run when predicting humor or offensive
# dataset = dataset[dataset['is_humor'] == 1]
dataset = dataset[dataset['humor_controversy'] == 1]

print(dataset.head(20))

   id                                               text  is_humor  \
0   1  TENNESSEE: We're the best state. Nobody even c...         1   
1   2  A man inserted an advertisement in the classif...         1   
2   3  How many men does it take to open a can of bee...         1   
3   4  Told my mom I hit 1200 Twitter followers. She ...         1   
4   5  Roses are dead. Love is fake. Weddings are bas...         1   

   humor_rating  humor_controversy  offense_rating  
0          2.42                1.0             0.2  
1          2.50                1.0             1.1  
2          1.95                0.0             2.4  
3          2.11                1.0             0.0  
4          2.78                0.0             0.1  
    id                                               text  is_humor  \
0    1  TENNESSEE: We're the best state. Nobody even c...         1   
1    2  A man inserted an advertisement in the classif...         1   
3    4  Told my mom I hit 1200 Twitter followers

In [68]:

X = dataset.iloc[:,1].values
print(X)
from sklearn.impute import SimpleImputer 
#print(humor_data.isnull().sum())
constant_imputer=SimpleImputer(strategy='constant', fill_value=0)

dataset.iloc[:]=constant_imputer.fit_transform(dataset)
Y = dataset.iloc[:,5].values

print(Y)

["TENNESSEE: We're the best state. Nobody even comes close. *Elevennessee walks into the room* TENNESSEE: Oh shit..."
 'A man inserted an advertisement in the classifieds "Wife Wanted". The next day, he received 1000 of replies, all reading: "You can have mine." Free delivery also available at your door step'
 "Told my mom I hit 1200 Twitter followers. She pointed out how my brother owns a house and I'm wanted by several collection agencies. Oh ma!"
 ...
 'I live in quite a pretentious area. Even the ducks demand butter with their bread.'
 'If you rearrange the letters of POSTMEN... They become VERY ANGRY.'
 "Sins are like viruses, it's better you keep them to yourself"]
[0.2 1.1 0.  ... 0.  0.4 0.1]


In [41]:
# only do when trying for is_humor or is_offensive
label_encoder_Y = LabelEncoder()
Y = label_encoder_Y.fit_transform(Y)
print(Y)

[161 167 123 ... 170  40   0]


In [69]:
train_examples, test_examples, train_labels, test_labels = train_test_split(X,Y, test_size=0.2,random_state=0)
print("Training entries: {}, test entries: {}".format(len(train_examples), len(test_examples)))
print("TRAIN Dataset -> ")
print(train_examples[:10])
print(" Test Dataset ->")
print(train_labels[:10])

Training entries: 1972, test entries: 493
TRAIN Dataset -> 
["I'm getting my girlfriend a prosthetic leg for Christmas It's a great stocking-filler."
 'Anybody that eats Tide pods is an idiot. They could go to Costco and get the generic brand for half the price.'
 "I left my Adderall in my Ford Fiesta. Now it's a @FordFocus"
 'Telescopes probably use mirrors which means there is absolutely no way to know how many vampires there are in space.'
 'When it comes to tipping cows, I always like to leave at least a 15% gratuity.'
 'A woman wakes up after a vaginal tuck to find three bunches of flowers beside her bed. One from her surgeon, to say all went well. One from her husband, "get well soon", and he loved her. One from Tommy in the burns unit, to say "Thank you for my new ears"'
 'What do you call an overweight homosexual? Jigglypuff.'
 "What's the hardest part about breaking up with a Japanese girl? You have to drop the bomb on her twice before she gets the point."
 "Sorry, Babe, it's 

# <font color="green">Build the model </font> <br>
The neural network is created by stacking layers—this requires three main architectural decisions:
 * How to represent the text?
 * How many layers to use in the model?
 * How many *hidden units* to use for each layer? 
In this example, the input data consists of sentences. The labels to predict are either 0 or 1.

One way to represent the text is to convert sentences into embeddings vectors. We can use a pre-trained text embedding as the first layer, which will have two advantages:
 *   we don't have to worry about text preprocessing,
 *   we can benefit from transfer learning.
 
For this example we will use a model from [TensorFlow Hub] (https://www.tensorflow.org/hub) called [google/tf2-preview/gnews-swivel-20dim/1](https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1).
There are three other models to test for the sake of this tutorial:
 * [google/tf2-preview/gnews-swivel-20dim-with-oov/1] (https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim-with-oov/1) - same as [google/tf2-preview/gnews-swivel-20dim/1] (https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1), but with 2.5% vocabulary converted to OOV buckets. This can help if vocabulary of the task and vocabulary of the model don't fully overlap.
 * [google/tf2-preview/nnlm-en-dim50/1] (https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1) - A much larger model with ~1M vocabulary size and 50 dimensions.
 * [google/tf2-preview/nnlm-en-dim128/1] (https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1) - Even larger model with ~1M vocabulary size and 128 dimensions.

Let's first create a Keras layer that uses a TensorFlow Hub model to embed the sentences, and try it out on a couple of input examples. Note that the output shape of the produced embeddings is a expected: `(num_examples, embedding_dimension)`.


In [70]:
model = "https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1"
hub_layer = hub.KerasLayer(model, output_shape=[50], input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 1.72691688e-01, -1.96662605e-01,  1.53206140e-01,
         3.01078483e-02,  3.61752331e-01,  8.19413960e-02,
         2.16661379e-01, -1.41651854e-01, -6.67500272e-02,
        -4.08735983e-02,  4.74083394e-01,  1.28887236e-01,
         1.65047333e-01,  1.02559201e-01, -5.40219620e-02,
        -2.26394057e-01,  2.50060298e-02,  2.27612212e-01,
        -1.43635914e-01, -4.06409055e-01, -7.06835687e-02,
        -2.16744497e-01, -1.23018719e-01, -1.13432370e-01,
        -1.54559404e-01,  9.25091282e-02, -4.58936930e-01,
         1.04611464e-01,  1.61529377e-01,  3.77314351e-02,
        -3.20078969e-01,  1.31589726e-01, -9.86229256e-02,
         8.90027285e-02, -1.37625471e-01,  9.96905938e-02,
         2.76223779e-01,  1.68240607e-01,  1.69724807e-01,
         5.58682904e-03,  3.67112011e-01, -1.56320259e-01,
        -7.63619915e-02, -9.91256982e-02, -4.08143729e-01,
        -1.38102518e-02, -6.91233203e-02,  1.59342177e-02,
       

<font color="green">Let's now build the full model: </font>

In [71]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_5 (KerasLayer)   (None, 50)                48190600  
_________________________________________________________________
dense_10 (Dense)             (None, 16)                816       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 17        
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


<font color="green"> The layers are stacked sequentially to build the classifier: </font> <br>
 
 1. The first layer is a TensorFlow Hub layer. This layer uses a pre-trained Saved Model to map a sentence into its embedding vector. The model that we are using ([google/tf2-preview/gnews-swivel-20dim/1](https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1)) splits the sentence into tokens, embeds each token and then combines the embedding. The resulting dimensions are: `(num_examples, embedding_dimension)`.
 2. This fixed-length output vector is piped through a fully-connected (`Dense`) layer with 16 hidden units.
 3. The last layer is densely connected with a single output node. This outputs logits: the log-odds of the true class, according to the model.

<font color="green"> Hidden units </font> <br>
 
The above model has two intermediate or "hidden" layers, between the input and output. 
The number of outputs (units, nodes, or neurons) is the dimension of the representational space for the layer. In other words, the amount of freedom the network is allowed when learning an internal representation.
 
If a model has more hidden units (a higher-dimensional representation space), and/or more layers, then the network can learn more complex representations. However, it makes the network more computationally expensive and may lead to learning unwanted patterns—patterns that improve performance on training data but not on the test data. This is called *overfitting*, and we'll explore it later.

<font color="green">Loss function and optimizer </font> <br>
 
A model needs a loss function and an optimizer for training. Since this is a binary classification problem and the model outputs a probability (a single-unit layer with a sigmoid activation), we'll use the `binary_crossentropy` loss function. 
 
This isn't the only choice for a loss function, you could, for instance, choose `mean_squared_error`. But, generally, `binary_crossentropy` is better for dealing with probabilities—it measures the "distance" between probability distributions, or in our case, between the ground-truth distribution and the predictions.
 
Later, when we are exploring regression problems (say, to predict the price of a house), we will see how to use another loss function called mean squared error.
 
<font color="green">Now, configure the model to use an optimizer and a loss function: </font>

In [72]:
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])


<font color="green">Create a validation set </font>

When training, we want to check the accuracy of the model on data it hasn't seen before. <br>
Create a *validation set* by setting apart 10,000 examples from the original training data. <br> 
(Why not use the testing set now? Our goal is to develop and tune our model using only the training data, then use the test data just once to evaluate our accuracy).

In [73]:
print(len(train_examples))

1972


In [74]:
x_val = train_examples[:1450]
partial_x_train = train_examples[1450:]

y_val = train_labels[:1450]
partial_y_train = train_labels[1450:]

<font color="green">Train the model </font> <br>
Train the model for 40 epochs in mini-batches of 512 samples. This is 40 iterations over all samples in the `x_train` and `y_train` tensors. While training, monitor the model's loss and accuracy on the 10,000 samples from the validation set:

In [75]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=15,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Epoch 1/15
2/2 [==============================] - 1s 607ms/step - loss: 0.6260 - accuracy: 0.0230 - val_loss: 0.6210 - val_accuracy: 0.0186
Epoch 2/15
2/2 [==============================] - 1s 634ms/step - loss: 0.5966 - accuracy: 0.0211 - val_loss: 0.6113 - val_accuracy: 0.0159
Epoch 3/15
2/2 [==============================] - 1s 528ms/step - loss: 0.5716 - accuracy: 0.0192 - val_loss: 0.6030 - val_accuracy: 0.0138
Epoch 4/15
2/2 [==============================] - 1s 652ms/step - loss: 0.5485 - accuracy: 0.0192 - val_loss: 0.5956 - val_accuracy: 0.0124
Epoch 5/15
2/2 [==============================] - 1s 532ms/step - loss: 0.5274 - accuracy: 0.0172 - val_loss: 0.5891 - val_accuracy: 0.0124
Epoch 6/15
2/2 [==============================] - 1s 539ms/step - loss: 0.5072 - accuracy: 0.0172 - val_loss: 0.5823 - val_accuracy: 0.0124
Epoch 7/15
2/2 [==============================] - 1s 598ms/step - loss: 0.4869 - accuracy: 0.0172 - val_loss: 0.5745 - val_accuracy: 0.0131
Epoch 8/15
2/2 [====

<font color="green"> Evaluate the model </font> <br>
And let's see how the model performs. Two values will be returned. <br>
-> Loss (a number which represents our error, lower values are better), and <br>
-> accuracy.

In [76]:
results = model.evaluate(test_examples, test_labels)

print("*******Codalab************",results)
print(test_labels)

16/16 [==============================] - 2s 129ms/step - loss: 0.4824 - accuracy: 0.0101
*******Codalab************ [0.4824236333370209, 0.010141988284885883]
[1.4  0.   0.3  0.2  0.2  0.5  0.05 0.35 0.   1.5  1.45 0.   0.   2.6
 1.95 3.25 2.55 0.25 0.2  2.45 0.2  0.2  0.25 0.55 0.   0.2  0.25 2.7
 0.   0.   0.1  0.5  2.85 0.2  0.1  1.65 1.8  0.   0.   0.   0.3  0.05
 0.   0.   0.65 4.1  0.15 2.85 0.1  2.7  3.45 1.15 0.05 0.7  0.25 0.2
 0.1  0.   0.15 0.05 0.5  1.3  0.05 0.1  0.35 0.2  2.85 0.95 3.15 1.05
 0.   1.25 0.2  3.85 0.55 0.25 0.1  0.15 0.05 1.45 0.45 2.7  1.7  3.1
 4.05 0.   0.   0.   0.6  0.   0.2  0.05 1.75 0.3  0.   2.   0.05 0.
 0.   0.1  2.15 0.05 2.35 3.65 0.15 1.1  0.5  3.8  0.   1.65 0.   3.55
 1.3  1.05 0.   0.   0.2  0.3  0.4  0.05 0.55 0.   0.   2.05 0.   0.
 0.   0.05 0.   0.   0.25 3.35 3.4  0.05 0.   0.4  0.15 0.25 2.25 0.
 3.6  2.45 0.   0.2  4.2  0.25 0.   0.   0.6  0.95 1.2  0.4  0.5  1.45
 0.35 0.25 0.85 2.55 0.2  0.3  0.15 2.25 0.   0.2  2.5  0.   0.05 1.65

In [77]:
codalab_data = pd.read_csv('public_dev.csv')
test_data_codalab = codalab_data['text'].values
results_codalab = model.predict(test_data_codalab)
print(results_codalab)

[[ 0.71463233]
 [ 0.466227  ]
 [ 0.43464866]
 [ 0.5293789 ]
 [ 0.6286638 ]
 [ 0.30118436]
 [ 0.6193511 ]
 [ 0.95704293]
 [ 0.7478211 ]
 [ 0.43221325]
 [ 0.83511585]
 [ 0.5363073 ]
 [ 0.4228049 ]
 [ 0.86665577]
 [ 0.43635595]
 [ 0.17332546]
 [ 0.53414094]
 [ 0.45308346]
 [ 0.6421553 ]
 [ 0.02027299]
 [ 0.38690934]
 [ 0.26311034]
 [ 0.87128294]
 [ 0.33200538]
 [ 0.5597373 ]
 [ 0.555234  ]
 [ 0.5751617 ]
 [ 0.5611706 ]
 [ 0.5191389 ]
 [ 0.44830987]
 [ 0.37261343]
 [ 0.58720833]
 [ 0.5899044 ]
 [ 0.7572112 ]
 [ 0.29644203]
 [ 1.0250517 ]
 [ 0.5845968 ]
 [ 0.33077806]
 [ 0.67161405]
 [ 0.5704586 ]
 [ 0.42673382]
 [ 0.4787372 ]
 [ 0.5068246 ]
 [ 0.2629834 ]
 [ 0.1727949 ]
 [ 0.7206178 ]
 [ 0.33040145]
 [ 0.51743907]
 [ 0.2683318 ]
 [ 0.70921516]
 [ 0.8130994 ]
 [ 0.40617532]
 [ 0.32237297]
 [ 0.51842517]
 [ 0.5116469 ]
 [ 0.5600267 ]
 [ 0.51031935]
 [ 0.46783584]
 [ 0.9689901 ]
 [ 0.5859816 ]
 [ 0.3487292 ]
 [ 0.23320568]
 [ 0.6203511 ]
 [ 0.7501102 ]
 [ 0.6104374 ]
 [ 0.80066204]
 [ 0.25312

In [78]:
for i in range(len(results_codalab)):
    print(results_codalab[i][0])

0.71463233
0.466227
0.43464866
0.5293789
0.6286638
0.30118436
0.6193511
0.95704293
0.7478211
0.43221325
0.83511585
0.5363073
0.4228049
0.86665577
0.43635595
0.17332546
0.53414094
0.45308346
0.6421553
0.020272987
0.38690934
0.26311034
0.87128294
0.33200538
0.5597373
0.555234
0.5751617
0.5611706
0.5191389
0.44830987
0.37261343
0.58720833
0.5899044
0.7572112
0.29644203
1.0250517
0.5845968
0.33077806
0.67161405
0.5704586
0.42673382
0.4787372
0.5068246
0.2629834
0.1727949
0.7206178
0.33040145
0.51743907
0.2683318
0.70921516
0.8130994
0.40617532
0.32237297
0.51842517
0.5116469
0.5600267
0.51031935
0.46783584
0.9689901
0.5859816
0.3487292
0.23320568
0.6203511
0.7501102
0.6104374
0.80066204
0.25312746
0.37512097
0.12652475
0.7649709
0.4499385
0.916028
0.61808854
0.97077876
1.5198821
0.48097432
0.7572388
0.32888997
0.53183204
0.30134413
0.524349
0.5404148
0.84546685
0.26987785
0.38601333
0.23680799
0.44103825
0.7507548
1.1322055
0.86374617
0.7391016
0.90799403
0.7332568
0.5814266
0.62877566
0.1